In [1]:
%pip install beautifulsoup4 requests selenium lxml openpyxl


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd

options = Options()
options.page_load_strategy = 'eager'
driver = webdriver.Chrome(options=options)

# 웹사이트에 접속합니다.
driver.get('https://data.kleague.com')

# Find frameset element and get its children
frames = driver.find_elements("tag name", 'frame')

for frame in frames:
    if 'https://portal.kleague.com' in frame.get_attribute('src'):
        # redirect the browser to the frame url
        driver.get(frame.get_attribute('src'))
    else:
        # do nothing
        pass

In [3]:
import re

def extract_onclick_data(onclick):
    if "moveMainFrameMcPlayer" in onclick:
        match = re.search(r"moveMainFrameMcPlayer\('(.+?)','(.+?)','(.+?)'\)", onclick)
        if match:
            return dict(zip(['menuCd', 'playerId', 'teamId'], match.groups()))
    elif "moveMainFrame" in onclick:
        match = re.search(r"moveMainFrame\('(.+?)'\)", onclick)
        if match:
            return {'menuCd': match.group(1)}
    return {}

In [4]:

# JavaScript를 실행하여 페이지를 이동합니다.
# driver.execute_script("moveMainFrame('0011')")
driver.execute_script("moveMainFrame('0415')")

In [5]:
# 페이지의 HTML을 가져옵니다.
html = driver.page_source

# BeautifulSoup 객체를 생성합니다.
soup = BeautifulSoup(html, 'html.parser')


# 특정 form 안에 있는 모든 테이블을 찾아 데이터를 스크랩핑합니다.
form = soup.find('form', {'id': 'frm'})
table_data = []
if form is not None:
    tables = form.find_all('table')
    for table in tables:
        # Get column names from thead
        column_names = [col.text for col in table.thead.find_all('th')]
        # Get all rows in the table
        rows = table.find_all('tr')
        row_data = []
        for row in rows:
            # Get all columns in the row
            cols = row.find_all('td')
            # Get onclick attribute value
            onclick = row.get('onclick')
            # Append the onclick value to the row data if it exists
            cols_data = dict(zip(column_names, [col.text for col in cols]))
            if onclick is not None:
                cols_data.update(extract_onclick_data(onclick))
            row_data.append(cols_data)
        table_data.append(pd.DataFrame(row_data))

# 데이터를 출력합니다.
for data in table_data:
    print(data)

# Export data to excel
with pd.ExcelWriter('kl_data.xlsx') as writer:
    for i, data in enumerate(table_data):
        data.to_excel(writer, sheet_name=f'Sheet{i+1}')

       소속 클럽   등번   성명      키   몸무게        생년월일 menuCd  playerId teamId
0        NaN  NaN  NaN    NaN   NaN         NaN    NaN       NaN    NaN
1       강원FC   41  김유성  187Cm  72Kg  2005/11/16   0416  20240099    K21
2       강원FC   21  박청효  190Cm  78Kg  1990/02/13   0416  20130134    K21
3       강원FC    1  이광연  184Cm  85Kg  1999/09/11   0416  20190058    K21
4       강원FC   71  조민규  193Cm  87Kg  2003/04/30   0416  20230068    K21
..       ...  ...  ...    ...   ...         ...    ...       ...    ...
99   포항 스틸러스   21  황인재  187Cm  73Kg  1994/04/22   0416  20160079    K03
100     화성FC   21  강성국  185Cm  76Kg  2005/03/11   0416  20250173    K39
101     화성FC   18  김기훈  186Cm  84Kg  2002/11/14   0416  20250172    K39
102     화성FC    1  김승건  189Cm  82Kg  1999/02/08   0416  20250166    K39
103     화성FC   13  이기현  192Cm  84Kg  1993/12/16   0416  20150056    K39

[104 rows x 9 columns]
    소속 클럽   등번   성명      키   몸무게        생년월일 menuCd  playerId teamId
0     NaN  NaN  NaN    NaN   NaN         Na